<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisPredictivo/blob/master/Kaggle/2023Q2/Un_primer_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import json,re
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
from preparando_datos import sum_into_column,split_and_sum,get_min_max
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt





In [14]:
df_val=pd.read_csv('dataset/testear.csv')
df_train=pd.read_csv('dataset/origen.csv')
#split the train dataset into train and test
#df_train, df_test = train_test_split(df_train, test_size=0.1, random_state=421)

df_list = [df_train,df_val]


In [15]:
for df in df_list:
    # if isAdult is larger than 1, then set it to 1, if smaller that 0, then set it to 0
    df['isAdult'] = df['isAdult'].apply(lambda x: 1 if x > 1 else x)
    df['seasonNumber'] = df['seasonNumber'].fillna(0)
    df['episodeNumber'] = df['episodeNumber'].fillna(1)
    df['ordering'] = df['ordering'].fillna(0)
    df['language'] = df['language'].fillna(0)


In [16]:
# Completando los 0 con los promedios tomando en cuenta si son series, peliculas o episodios
for df in df_list:
    df['runtimeMinutes'] = df['runtimeMinutes'].replace(0, np.nan)
    df['runtimeMinutes'] = df.groupby('titleType')['runtimeMinutes'].apply(lambda x: x.interpolate(method='linear'))

    #Lo mismo pero para el budget
    df['budget'] = df['budget'].replace(0, np.nan)
    df['budget'] = df.groupby('titleType')['budget'].apply(lambda x: x.interpolate(method='linear'))
    
    # Si endYear es 0, entonces se le asigna el valor de startYear
    df['endYear'] = df.apply(lambda x: x['startYear'] if x['endYear'] == 0 else x['endYear'], axis=1)

    # Lo mismo pero para revenue
    df['revenue'] = df['revenue'].replace(0, np.nan)
    df['revenue'] = df.groupby('titleType')['revenue'].apply(lambda x: x.interpolate(method='linear'))
    

C:\Users\ianda\AppData\Local\Temp\ipykernel_31308\3645156677.py:4: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df['runtimeMinutes'] = df.groupby('titleType')['runtimeMinutes'].apply(lambda x: x.interpolate(method='linear'))
C:\Users\ianda\AppData\Local\Temp\ipykernel_31308\3645156677.py:8: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, u

In [17]:
# Seasons worked

for df in df_list:
    
    df["dir_qty"] = pd.Series(df["directors"]).str.split(",").apply(len)

    writers = df.groupby("directors").count()["Unnamed: 0"].to_dict()
    writers = split_and_sum(writers)
    df["directors_exp"] = df["directors"].apply(sum_into_column,args=(writers,))

    df["dir_minExperience"] = df["directors"].apply(get_min_max,args=(writers,0,))

    df["dir_maxExperience"] = df["directors"].apply(get_min_max,args=(writers,1,))

    seasons = df.groupby("directors").count()["seasonNumber"].to_dict()
    seasons = split_and_sum(seasons)
    df["dir_seasonsOfExperience"] = df["directors"].apply(sum_into_column,args=(seasons,))

    episodes = df.groupby("directors").count()["episodeNumber"].to_dict()
    episodes = split_and_sum(episodes)
    df["dir_episodesOfExperience"] = df["directors"].apply(sum_into_column,args=(episodes,))

    movies = df[df["titleType"] == "movie"].groupby("directors").count()["titleType"].to_dict()
    movies = split_and_sum(movies)
    df["dir_moviesOfExperience"] = df["directors"].apply(sum_into_column,args=(movies,))

    others = df[df["titleType"] != "movie"].groupby("directors").count()["titleType"].to_dict()
    others = split_and_sum(others)
    df["dir_othersOfExperience"] = df["directors"].apply(sum_into_column,args=(others,))

    
    
    

del seasons,episodes,movies,others,df

In [18]:
for df in df_list:
    df["writers_qty"] = pd.Series(df["writers"]).str.split(",").apply(len)

    writers = df.groupby("writers").count()["Unnamed: 0"].to_dict()
    writers = split_and_sum(writers)
    df["writers_exp"] = df["writers"].apply(sum_into_column,args=(writers,))

    df["writer_minExperience"] = df["writers"].apply(get_min_max,args=(writers,0,))

    df["writer_maxExperience"] = df["writers"].apply(get_min_max,args=(writers,1,))


    seasons = df.groupby("writers").count()["seasonNumber"].to_dict()
    seasons = split_and_sum(seasons)
    df["writer_seasonsOfExperience"] = df["writers"].apply(sum_into_column,args=(seasons,))

    episodes = df.groupby("writers").count()["episodeNumber"].to_dict()
    episodes = split_and_sum(episodes)
    df["writer_episodesOfExperience"] = df["writers"].apply(sum_into_column,args=(episodes,))

    movies = df[df["titleType"] == "movie"].groupby("writers").count()["titleType"].to_dict()
    movies = split_and_sum(movies)
    df["writer_moviesOfExperience"] = df["writers"].apply(sum_into_column,args=(movies,))

    others = df[df["titleType"] != "movie"].groupby("writers").count()["titleType"].to_dict()
    others = split_and_sum(others)
    df["writer_othersOfExperience"] = df["writers"].apply(sum_into_column,args=(others,))


del seasons,episodes,movies,others,df,writers

In [19]:
def parse_json(x):
    x = x.split(":")
    x = x[1:]
    for i in range(len(x)):
        x[i] = x[i].split("}")[0]
        x[i] = x[i].split(",")[0]
    
    #split the list into pairs of two
    x = [x[i:i+2] for i in range(0,len(x),2)]
    # grab the first element of the pair
    x = [i[0] for i in x]
    #remove the quotes
    x = [i.replace("'",'') for i in x]
    #remove the spaces
    x = [i.strip() for i in x]

    return ",".join(x)
for df in df_list:
    #turn production company into a list
    #fill nans with empty list
    df["production_countries"] = df["production_countries"].fillna("[]")
    #turn str list into list
    df["production_countries"] = df["production_countries"].apply(parse_json)
    experience = split_and_sum(df.groupby("production_countries").count()["Unnamed: 0"].to_dict())
    experience['']=0
    df["production_countries_experience"] = df["production_countries"].apply(sum_into_column,args=(experience,))

    df['production_countries'] = df['production_countries'].apply(lambda x: len(x.split(',')))

    #turn production company into a list
    #fill nans with empty list
    df["production_companies"] = df["production_companies"].fillna("[]")
    #turn str list into list
    df["production_companies"] = df["production_companies"].apply(parse_json)
    experience = split_and_sum(df.groupby("production_companies").count()["Unnamed: 0"].to_dict())
    experience['']=0
    df["production_companies_experience"] = df["production_companies"].apply(sum_into_column,args=(experience,))

    df['production_companies'] = df['production_companies'].apply(lambda x: len(x.split(',')))
    dir_votes = df.groupby("directors").sum()["numVotes"].to_dict()
    dir_votes = split_and_sum(dir_votes)
    df["dir_votes"] = df["directors"].apply(sum_into_column,args=(dir_votes,))
    
    

C:\Users\ianda\AppData\Local\Temp\ipykernel_31308\2738369730.py:40: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dir_votes = df.groupby("directors").sum()["numVotes"].to_dict()
C:\Users\ianda\AppData\Local\Temp\ipykernel_31308\2738369730.py:40: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dir_votes = df.groupby("directors").sum()["numVotes"].to_dict()


In [20]:
# turn video colunm into 1 or 0
# video column might have False, True or NaN values
for df in df_list:
    df["video"] = df["video"].fillna(False)
    df["video"] = df["video"].astype(int)
    df["tagline"] = df["tagline"].fillna("")
    df["tagline"] = df["tagline"].apply(len)



In [21]:
y=df_train.averageRating
vars=['startYear', 'runtimeMinutes',"numVotes","writers_exp",
      "directors_exp","endYear","isAdult","seasonNumber",
      "episodeNumber","ordering","isOriginalTitle",
      "popularity","budget","dir_seasonsOfExperience",
      "dir_episodesOfExperience","dir_othersOfExperience","dir_moviesOfExperience",
      "writer_seasonsOfExperience","writer_episodesOfExperience","writer_othersOfExperience",
      "writer_moviesOfExperience","video","tagline","production_countries","production_countries_experience",
      "dir_votes",'production_companies', 'production_companies_experience',"writers_qty","dir_qty","dir_minExperience","dir_maxExperience",
      "writer_minExperience","writer_maxExperience"]
X_pred=df_val[vars]
vars.append("averageRating")
X=df_train[vars]
#X_test=df_test[vars]

for db in [[X,df_train],[X_pred,df_val]]:
      db[0] = pd.concat([db[0], pd.get_dummies(db[1]['language'], prefix='language')], axis=1)
      db[0] = pd.concat([db[0], pd.get_dummies(db[1]['status'], prefix='status')], axis=1)

del db



In [22]:
for i,df in enumerate(df_list):
    encoder = OneHotEncoder()
    title = encoder.fit_transform(df[['titleType']])
    title_df = pd.DataFrame(title.toarray(), columns=encoder.get_feature_names_out(['titleType']))
    #split by genres
    genres = df.genres_x.str.get_dummies(sep=',')
    genres = genres.reindex(sorted(genres.columns), axis=1)

    title_df = pd.concat([title_df, genres], axis=1)
    if i==0:
        X = pd.concat([X, title_df], axis=1)
    elif i==1:
        X_pred = pd.concat([X_pred, title_df], axis=1)
    print(i,"Done!")
    


del title_df, genres, title, encoder, df,i





0 Done!
1 Done!


In [23]:
# Drop duplicate columns
X = X.loc[:, ~X.columns.duplicated()]
X_pred = X_pred.loc[:, ~X_pred.columns.duplicated()]


# Add missing columns to X_pred and X_test
missing_cols = set(X.columns) - set(X_pred.columns)
X_pred = X_pred.reindex(columns=X_pred.columns.tolist() + list(missing_cols))


# Add missing columns to X
missing_cols = set(X_pred.columns) - set(X.columns)
X = X.reindex(columns=X.columns.tolist() + list(missing_cols))


In [24]:
X_pred = X_pred.sort_index(axis=1)
X = X.sort_index(axis=1)


#Drop averageRating from X_pred
X_pred = X_pred.drop(columns=["averageRating"])

X.to_csv('dataset/train.csv', index=False)
X_pred.to_csv('dataset/val.csv', index=False)
